In [1]:
! python -m pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 42.8 MB/s eta 0:00:00


In [4]:
from gurobipy import Model, GRB, quicksum

# Insiemi

I = ["auto", "moto", "bicicletta", "monopattino"]   # prodotti
J = ["ruote", "tubi", "bulloni"]                    # materie prime

# Parametri

P = {
    "auto": 5000,
    "moto": 2000,
    "bicicletta": 300,
    "monopattino": 150,
}

C = {
    "auto": 3000,
    "moto": 1200,
    "bicicletta": 120,
    "monopattino": 60,
}

F = {
    "auto": 10000,
    "moto": 5000,
    "bicicletta": 1000,
    "monopattino": 500,
}

B = {
    "ruote": 400,
    "tubi": 500,
    "bulloni": 5000,
}

A = {
    ("auto", "ruote"): 4,
    ("auto", "tubi"): 5,
    ("auto", "bulloni"): 50,

    ("moto", "ruote"): 2,
    ("moto", "tubi"): 3,
    ("moto", "bulloni"): 30,

    ("bicicletta", "ruote"): 2,
    ("bicicletta", "tubi"): 2,
    ("bicicletta", "bulloni"): 15,

    ("monopattino", "ruote"): 2,
    ("monopattino", "tubi"): 1,
    ("monopattino", "bulloni"): 10,
}


W = 200000

M = W

# Implementazione modello
model = Model("Produzione")

## Variabili decisionali
x = model.addVars(I, vtype=GRB.INTEGER, lb=0, name="x")
y = model.addVars(I, vtype=GRB.BINARY, name="y")


## Vincoli

### (1)
for j in J:
    model.addConstr(
        quicksum(A[i, j] * x[i] for i in I) <= B[j],
        name=f"disponibilita_{j}"
    )

### (2)
model.addConstr(
    quicksum(C[i] * x[i] + F[i] * y[i] for i in I) <= W,
    name="budget"
)

### (3)
for i in I:
    model.addConstr(
        x[i] <= M * y[i],
        name=f"attivazione_{i}"
    )

## funzione obiettivo
model.setObjective(quicksum(P[i] * x[i] for i in I), GRB.MAXIMIZE)


# Ottimizza
model.optimize()

# Stampa soluzione
if model.status == GRB.OPTIMAL:
    print(f"Profitto totale: {model.objVal:.2f}")
    print("Produzione:")
    for i in I:
        qty = x[i].X
        if qty > 0.5:
            print(f"  {i}: x = {qty:.0f}, y = {int(round(y[i].X))}")

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 8 columns and 28 nonzeros (Max)
Model fingerprint: 0x18d22aad
Model has 4 linear objective coefficients
Variable types: 0 continuous, 8 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [2e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 2e+05]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 306200.00000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 8 columns, 24 nonzeros
Variable types: 0 continuous, 8 integer (4 binary)

Root relaxation: objective 3.264668e+05, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexp